In [2]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
""" Import data """
trade_df = pd.read_csv('data/trade_data.csv')
alliance_df = pd.read_csv('data/alliance_data.csv')
common_iso_codes = pd.read_csv('data/common_iso_codes.csv')
trade_df['alliance'] = pd.Series()

In [4]:
""" Convert CCode into iso code """
alliance_df['iso3A'] = alliance_df['stateA'].map(dict(zip(common_iso_codes['CCode'], common_iso_codes['iso3'])))
alliance_df['iso3B'] = alliance_df['stateB'].map(dict(zip(common_iso_codes['CCode'], common_iso_codes['iso3'])))

In [5]:
trade_df.head()

,reporter,partner,year,export,import,alliance
0,AFG,AUS,2008,10.692,129.485,NaN
1,AFG,AUS,2009,8.462,1214.034,NaN
2,AFG,AUS,2010,648.457,1329.447,NaN
3,AFG,AUS,2011,111.502,1301.370,NaN
4,AFG,AUS,2015,1296.260,1259.046,NaN


In [6]:
alliance_df.head()

,state_nameA,state_nameB,year,atopally,defense,offense,neutral,nonagg,consul,shareob,...,bilatno,multino,number,asymm,stateA,stateB,version,atopids,iso3A,iso3B
0,United States of America,Canada,1942,1,1,1,0,0,0,NaN,...,0,1,1,0,2,20,5.1,[2550.0],USA,CAN
1,United States of America,Canada,1943,1,1,1,0,0,0,NaN,...,0,1,1,0,2,20,5.1,[2550.0],USA,CAN
2,United States of America,Canada,1944,1,1,1,0,0,0,NaN,...,0,1,1,0,2,20,5.1,[2550.0],USA,CAN
3,United States of America,Canada,1945,1,1,1,0,0,0,NaN,...,0,1,1,0,2,20,5.1,[2550.0],USA,CAN
4,United States of America,Canada,1949,1,1,0,0,1,1,NaN,...,0,1,1,0,2,20,5.1,[3180.0],USA,CAN


In [7]:
""" Generating trade and alliance networks """

trade_networks = {}
for i in trade_df.groupby('year'):
    year = i[0]
    df = i[1]
    G = nx.from_pandas_edgelist(df, 'reporter', 'partner', 'export', create_using=nx.DiGraph())
    trade_networks[year] = G

alliance_networks = {}
for j in alliance_df.groupby('year'):
    year = j[0]
    df = j[1]
    G = nx.from_pandas_edgelist(df, 'state_nameA', 'state_nameB', 'defense', create_using=nx.DiGraph())
    alliance_networks[year] = G

In [8]:
nx.draw(trade_networks[0][-1], with_labels=True, pos=nx.spring_layout(trade_networks[0][-1]))

KeyError: 0

In [73]:
alliance_communities = {}
for year, graph in alliance_networks.items():
    alliance_communities[year] = nx.algorithms.community.louvain_communities(graph)

trade_communities = {}
for year, graph in trade_networks.items():
    trade_communities[year] = nx.algorithms.community.louvain_communities(graph)

In [74]:
# Alliance communities for a given year
print(alliance_communities[2018])

print()

# Trade communities for a given year
print(trade_communities[2018])

[{'Belize', 'Barbados', 'Mexico', 'St. Lucia', 'Venezuela', 'Costa Rica', 'Bolivia', 'Haiti', 'Guyana', 'Colombia', 'St. Kitts and Nevis', 'Uruguay', 'Brazil', 'Guatemala', 'Ecuador', 'Dominican Republic', 'Panama', 'United States of America', 'Tuvalu', 'Jamaica', 'Chile', 'Trinidad and Tobago', 'Grenada', 'Dominica', 'El Salvador', 'Suriname', 'Paraguay', 'Argentina', 'Antigua & Barbuda', 'Nicaragua', 'St. Vincent and the Grenadines', 'Honduras', 'Peru', 'Afghanistan', 'Kiribati', 'Bahamas'}, {'Estonia', 'Croatia', 'Monaco', 'Turkey', 'Belgium', 'Lithuania', 'Bulgaria', 'Italy', 'Uzbekistan', 'Poland', 'Portugal', 'Kyrgyzstan', 'Turkmenistan', 'Denmark', 'Yugoslavia', 'Slovakia', 'San Marino', 'Kazakhstan', 'Armenia', 'Liechtenstein', 'Romania', 'Czech Republic', 'Finland', 'Greece', 'Norway', 'Austria', 'Latvia', 'Spain', 'Belarus', 'Bosnia and Herzegovina', 'Switzerland', 'Mongolia', 'Sweden', 'Cyprus', 'Moldova', 'Canada', 'Tajikistan', 'Netherlands', 'Hungary', 'Azerbaijan', 'Mont

In [75]:
""" Compare trade and alliance communities """
for year in range(1988,2019):
    trade_comm = trade_communities[year]
    alliance_comm = alliance_communities[year]
    print(f'Year: {year}')
    print(f'Number of trade communities: {len(trade_comm)}')
    print(f'Number of alliance communities: {len(alliance_comm)}\n')

Year: 1988
Number of trade communities: 3
Number of alliance communities: 5

Year: 1989
Number of trade communities: 3
Number of alliance communities: 5

Year: 1990
Number of trade communities: 2
Number of alliance communities: 6

Year: 1991
Number of trade communities: 3
Number of alliance communities: 6

Year: 1992
Number of trade communities: 4
Number of alliance communities: 7

Year: 1993
Number of trade communities: 3
Number of alliance communities: 6

Year: 1994
Number of trade communities: 4
Number of alliance communities: 5

Year: 1995
Number of trade communities: 4
Number of alliance communities: 5

Year: 1996
Number of trade communities: 4
Number of alliance communities: 6

Year: 1997
Number of trade communities: 4
Number of alliance communities: 5

Year: 1998
Number of trade communities: 4
Number of alliance communities: 6

Year: 1999
Number of trade communities: 4
Number of alliance communities: 6

Year: 2000
Number of trade communities: 4
Number of alliance communities: 4


In [76]:
""" Find common alliances """

for year in range(2018,2019):
    trade_year = trade_df[trade_df['year'] == year]
    alliance_year = alliance_df[alliance_df['year'] == year]

    for dyad in trade_year[['reporter', 'partner']].values:
        dyad_has_alliance = alliance_year.loc[(alliance_year['iso3A'] == dyad[0]) & (alliance_year['iso3B'] == dyad[1])]
        if not dyad_has_alliance.empty:
            trade_year.loc[(trade_year['reporter'] == dyad[0]) & (trade_year['partner'] == dyad[1])
            ]['alliance'] = 1

    trade_df.loc[trade_df['year'] == year] = trade_year

C:\Users\AKaiz\AppData\Local\Temp\ipykernel_1040\1257793532.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade_year.loc[(trade_year['reporter'] == dyad[0]) & (trade_year['partner'] == dyad[1])


In [79]:
trade_df

,reporter,partner,year,export,import,alliance
0,AFG,AUS,2008,10.692000,129.485000,NaN
1,AFG,AUS,2009,8.462000,1214.034000,NaN
2,AFG,AUS,2010,648.457000,1329.447000,NaN
3,AFG,AUS,2011,111.502000,1301.370000,NaN
4,AFG,AUS,2015,1296.260000,1259.046000,NaN
...,...,...,...,...,...,...
729896,ZWE,ZMB,2017,58352.517720,125750.699130,NaN
729897,ZWE,ZMB,2018,66580.106170,176819.220710,NaN
729898,ZWE,ZMB,2019,59552.557370,96423.163390,NaN
729899,ZWE,ZMB,2020,52563.443335,111427.350996,NaN
